# Main 

In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import pandas as pd
import numpy as np
import requests
import polyline
from dash.exceptions import PreventUpdate

# Load data with error handling
try:
    df = pd.read_excel('AT_BikeLanes_08282024.xlsx')
except Exception as e:
    print(f"Error loading the data: {e}")
    df = pd.DataFrame()  # Empty DataFrame if loading fails

# Clean the dataset
def clean_data(df):
    # Handle Missing Values
    df = df.dropna(subset=['StartPoint(Y)', 'StartPoint(X)', 'ID', 'RoadSection', 'TypeofWorks'])
    df.loc[:, 'Length(km)'] = df['Length(km)'].fillna(0)
    
    # Remove Duplicates
    df = df.drop_duplicates()
    
    # Data Type Conversion
    df['Length(km)'] = df['Length(km)'].astype(float)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Coordinate Validation
    df = df[(df['StartPoint(Y)'].between(-90, 90)) & (df['StartPoint(X)'].between(-180, 180))].copy()
    return df

# Clean the DataFrame
df = clean_data(df)

# Get unique RoadSection names for the dropdown
road_sections = sorted(df['RoadSection'].unique())

# Define bounds for the Philippines
PHILIPPINES_BOUNDS = {
    'southwest': [4.64, 116.95],
    'northeast': [21.19, 126.60]
}

# OSRM server URL
OSRM_SERVER = "http://router.project-osrm.org"

def get_route(start_coord, end_coord):
    """Get a route between two points using OSRM"""
    url = f"{OSRM_SERVER}/route/v1/biking/{start_coord[1]},{start_coord[0]};{end_coord[1]},{end_coord[0]}?overview=full&geometries=polyline"
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        route = response.json()['routes'][0]['geometry']
        return polyline.decode(route)
    except requests.exceptions.RequestException as e:
        print(f"Error getting route: {e}")
        return None

# Create a Dash app
app = dash.Dash(__name__)

# Function to create markers and polylines
# Function to create markers and polylines
def create_markers_and_polylines():
    markers = []
    polylines = []
    
    # Create markers and polylines for all data points
    for index, row in df.iterrows():
        start_lat, start_lon = row['StartPoint(Y)'], row['StartPoint(X)']
        end_lat, end_lon = row['EndPoint(Y)'], row['EndPoint(X)']
        if (PHILIPPINES_BOUNDS['southwest'][0] <= start_lat <= PHILIPPINES_BOUNDS['northeast'][0]) and \
           (PHILIPPINES_BOUNDS['southwest'][1] <= start_lon <= PHILIPPINES_BOUNDS['northeast'][1]) and \
           pd.notnull(start_lat) and pd.notnull(start_lon) and pd.notnull(end_lat) and pd.notnull(end_lon):
            
            # Start point marker
            start_marker = dl.Marker(
                position=[start_lat, start_lon],
                id={'type': 'start_marker', 'index': index},
                children=[
                    dl.Tooltip(f"{row['RoadSection']} (Start)"),
                    dl.Popup(
                        html.Div([
                            html.P(f"ID: {row['ID']}"),
                            html.P(f"Type of Works: {row['TypeofWorks']}"),
                            html.P(f"Length: {row['Length(km)']} km"),
                            html.P(f"Class: {row['Class']}")
                        ])
                    )
                ]
            )
            markers.append(start_marker)

            # End point marker
            end_marker = dl.Marker(
                position=[end_lat, end_lon],
                id={'type': 'end_marker', 'index': index},
                children=[
                    dl.Tooltip(f"{row['RoadSection']} (End)"),
                    dl.Popup(
                        html.Div([
                            html.P(f"ID: {row['ID']}"),
                            html.P(f"Type of Works: {row['TypeofWorks']}"),
                            html.P(f"Length: {row['Length(km)']} km"),
                            html.P(f"Class: {row['Class']}")
                        ])
                    )
                ]
            )
            markers.append(end_marker)
            
            route = get_route([start_lat, start_lon], [end_lat, end_lon])
            if route:
                polyline = dl.Polyline(
                    positions=route,
                    color="blue",
                    weight=3,
                    opacity=0.6,
                    id={'type': 'polyline', 'index': index}
                )
                polylines.append(polyline)
    
    return markers + polylines

# Layout of the app
app.layout = html.Div([
    html.H1("Bike Lanes Dashboard"),
    dcc.Store(id='zoom-level', data=6),
    dcc.Store(id='selected-line', data=None),
    html.Div([
        dcc.Dropdown(
            id='road-section-dropdown',
            options=[{'label': i, 'value': i} for i in road_sections],
            placeholder='Search for a Road Section',
            style={'width': '50%'}
        ),
        html.Button('Go', id='search-button', n_clicks=0)
    ]),
    dl.Map(
        center=[12.8797, 121.7740],
        zoom=6,
        bounds=[PHILIPPINES_BOUNDS['southwest'], PHILIPPINES_BOUNDS['northeast']],
        children=[
            dl.TileLayer(url="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png", attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'),
            dl.LayerGroup(id='marker-layer', children=create_markers_and_polylines())
        ],
        style={'width': '1000px', 'height': '800px'},
        id='bike-lanes-map'
    ),
    html.Div(id='loading', children="Loading...", style={'display': 'none'}),
    html.Div([
        html.H3("Bike Lane Details"),
        html.Div(id='bike-lane-info')
    ])
])

# Callbacks
@app.callback(
    Output('bike-lanes-map', 'center'),
    Output('bike-lanes-map', 'zoom'),
    Input('search-button', 'n_clicks'),
    State('road-section-dropdown', 'value'),
    prevent_initial_call=True
)
def search_road_section(n_clicks, selected_road):
    if not selected_road:
        raise PreventUpdate

    matching_row = df[df['RoadSection'] == selected_road]
    if matching_row.empty:
        print(f"No matching road section found for: {selected_road}")
        return [12.8797, 121.7740], 6  # Default center and zoom

    lat = matching_row.iloc[0]['StartPoint(Y)']
    lon = matching_row.iloc[0]['StartPoint(X)']

    if pd.isna(lat) or pd.isna(lon):
        print(f"Invalid coordinates for road section: {selected_road}")
        return [12.8797, 121.7740], 6  # Default center and zoom

    print(f"Centering map on: {selected_road} at coordinates: {lat}, {lon}")
    return [lat, lon], 15  # Center on the found location and zoom in

@app.callback(
    Output('road-section-dropdown', 'options'),
    Input('road-section-dropdown', 'search_value')
)
def update_options(search_value):
    if not search_value:
        return [{'label': i, 'value': i} for i in road_sections[:100]]  # Return first 100 options if no search
    return [{'label': i, 'value': i} for i in road_sections if search_value.lower() in i.lower()]

if __name__ == '__main__':
    app.run_server(debug=True)

Centering map on: E.O PEREZ at coordinates: 10.32047676, 123.9260313
Centering map on: E.O PEREZ at coordinates: 10.32047676, 123.9260313
Centering map on: A.C CORTEZ at coordinates: 10.32279866, 123.9540366
Centering map on: A.C CORTEZ at coordinates: 10.32279866, 123.9540366


# TEST